- Labels generated by financial sentiment analysis

# Trading Signal Generation

In [1]:
# Core Libraries
import pandas as pd
import numpy as np
import re

# Text Processing
import spacy
from transformers import AutoTokenizer, AutoModel

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm as notebook_tqdm

# Model Saving
import joblib

c:\Users\scaro\Downloads\pt-br\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preprocessing

In [2]:
file_path = "data/processed/labeled_january_data.csv"

with open(file_path, "r", encoding="utf-8") as file:
    df_jan = pd.read_csv(file)

In [ ]:
sns.countplot(x='label', data=df_jan)
plt.title('Label Distribution in January Dataset')
plt.savefig("results/bert_embeddings_experiment_v1/figures/label_distribution_january.png")
plt.show()

In [ ]:
file_path = "data/processed/labeled_february_data.csv"

with open(file_path, "r", encoding="utf-8") as file:
    df_feb = pd.read_csv(file)

In [ ]:
sns.countplot(x='label', data=df_feb)
plt.title('Label Distribution in February Dataset')
plt.savefig("results/bert_embeddings_experiment_v1/figures/label_distribution_february.png")
plt.show()

In [4]:
file_path = "data/processed/labeled_march_data.csv"

with open(file_path, "r", encoding="utf-8") as file:
    df_march = pd.read_csv(file)

In [ ]:
sns.countplot(x='label', data=df_march)
plt.title('Label Distribution in March Dataset')
plt.savefig("results/bert_embeddings_experiment_v1/figures/label_distribution_march.png")
plt.show()

In [5]:
import re

def normalize_numbers(text):
    # Replace monetary values (e.g., "R$ 15 bilhões" → "15B")
    text = re.sub(r"R\$ ?([\d.,]+) bilhões", r"\1B", text)
    text = re.sub(r"R\$ ?([\d.,]+) milhões", r"\1M", text)
    # Standardize percentages (e.g., "0.5 pp" → "0.5%")
    text = re.sub(r"([\d.,]+) pp", r"\1%", text)
    # Standardize plain numbers (e.g., "15,000" → "15000")
    text = text.replace(",", "")
    return text

In [6]:
def remove_noise(text):
    # Remove dates and times
    text = re.sub(r"\b\d{1,2}[hH]\d{2}\b", "", text)  # Times like "17h20"
    text = re.sub(r"\b\d{1,2}ª[Ff]\b", "", text)       # Ordinals like "2ªF"
    # Remove redundant words
    text = re.sub(r"MAIS AGENDA|LÁ FORA|A BOLSA ESTÁ CARA", "", text, flags=re.IGNORECASE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [7]:
import re

# Comprehensive dictionary for expanding acronyms
acronyms = {
    "Selic": "Sistema Especial de Liquidação e de Custódia",
    "PIB": "Produto Interno Bruto",
    "CDI": "Certificado de Depósito Interbancário",
    "LPRs": "Loan Prime Rates",
    "Ibovespa": "Índice Bovespa",
    "BB": "Banco do Brasil",
    "BC": "Banco Central",
    "FGTS": "Fundo de Garantia do Tempo de Serviço",
    "STF": "Supremo Tribunal Federal",
    "CPI": "Índice de Preços ao Consumidor",
    "MP": "Medida Provisória",
    "EUA": "Estados Unidos",
    "ONU": "Organização das Nações Unidas",
    "FGV": "Fundação Getúlio Vargas",
    "IBGE": "Instituto Brasileiro de Geografia e Estatística",
    "BNDES": "Banco Nacional de Desenvolvimento Econômico e Social",
    "IPCA": "Índice Nacional de Preços ao Consumidor Amplo",
    "DI": "Depósito Interfinanceiro",
    "IR": "Imposto de Renda",
    "OI": "Operadora Oi",
    "CV": "Câmara de Vereadores"
}

noisy_acronyms = {"ROMI", "ENEVA", "LIGHT", "DA"}

def expand_acronyms(text, acronym_dict):
    for acronym, full_form in acronym_dict.items():
        text = re.sub(rf'\b{re.escape(acronym)}\b', full_form, text, flags=re.IGNORECASE)
    return text

def remove_noisy_acronyms(text, noisy_set):
    return re.sub(r'\b(?:' + '|'.join(noisy_set) + r')\b', '', text)


In [8]:
import spacy

# Load Portuguese spaCy model
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

In [9]:
def preprocess_text(text):
    text = normalize_numbers(text)         
    text = expand_acronyms(text, acronyms)
    text = remove_noisy_acronyms(text, noisy_acronyms)
    text = lemmatize_text(text)          
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

# Apply preprocessing to the dataset
df_jan['cleaned_article'] = df_jan['article'].apply(
    lambda x: pd.Series(preprocess_text(x))
)
df_feb['cleaned_article'] = df_feb['article'].apply(
    lambda x: pd.Series(preprocess_text(x))
)
df_march['cleaned_article'] = df_march['article'].apply(
    lambda x: pd.Series(preprocess_text(x))
)

## Sentiment Analysis

In [10]:
from transformers import AutoTokenizer, BertForSequenceClassification
import os
import pandas as pd
import re
from IPython.display import display

In [11]:
from transformers import (
    AutoTokenizer, 
    BertForSequenceClassification,
    pipeline,
)

tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbertptbr = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")

In [12]:
classification_mapper = {
    0: 1,  # POSITIVE -> +1
    1: -1, # NEGATIVE -> -1
    2: 0   # NEUTRAL -> 0
}

In [13]:
def classify_article(article):
    try:
        # Tokenize the article
        tokens = tokenizer([article], return_tensors="pt", padding=True, truncation=True, max_length=512)
        # Get model outputs
        outputs = finbertptbr(**tokens)
        # Map predictions to classifications
        pred_index = np.argmax(outputs.logits.cpu().detach().numpy(), axis=1)[0]
        return classification_mapper[pred_index]
    except Exception as e:
        print(f"Error processing article: {article} | Error: {e}")
        return None

In [14]:
# Apply the classification function to each DataFrame
df_jan['classification'] = df_jan['cleaned_article'].apply(classify_article)
df_feb['classification'] = df_feb['cleaned_article'].apply(classify_article)
df_march['classification'] = df_march['cleaned_article'].apply(classify_article)

In [27]:
import os
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
def save_classification_report(y_true, y_pred, filepath):
    report = classification_report(y_true, y_pred, labels=[1, 0, -1], target_names=["Positive (+1)", "Neutral (0)", "Negative (-1)"])
    with open(filepath, "w") as f:
        f.write(report)
    print(f"Classification report saved to {filepath}")

def save_confusion_matrix(y_true, y_pred, filepath):
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0, -1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Positive (+1)", "Neutral (0)", "Negative (-1)"])
    disp.plot(cmap="Blues", values_format="d")
    plt.title("Confusion Matrix")
    plt.savefig(filepath)
    plt.close()
    print(f"Confusion matrix saved to {filepath}")

output_dir = "results/sentiment_analysis_experiment/metrics"
os.makedirs(output_dir, exist_ok=True)

In [30]:
# January
save_classification_report(
    df_jan['label'], 
    df_jan['classification'], 
    os.path.join(output_dir, "january_classification_report.txt")
)
save_confusion_matrix(
    df_jan['label'], 
    df_jan['classification'], 
    os.path.join(output_dir, "january_confusion_matrix.png")
)

# February
save_classification_report(
    df_feb['label'], 
    df_feb['classification'], 
    os.path.join(output_dir, "february_classification_report.txt")
)
save_confusion_matrix(
    df_feb['label'], 
    df_feb['classification'], 
    os.path.join(output_dir, "february_confusion_matrix.png")
)

# March
save_classification_report(
    df_march['label'], 
    df_march['classification'], 
    os.path.join(output_dir, "march_classification_report.txt")
)
save_confusion_matrix(
    df_march['label'], 
    df_march['classification'], 
    os.path.join(output_dir, "march_confusion_matrix.png")
)

Classification report saved to results/sentiment_analysis_experiment/metrics\january_classification_report.txt
Confusion matrix saved to results/sentiment_analysis_experiment/metrics\january_confusion_matrix.png
Classification report saved to results/sentiment_analysis_experiment/metrics\february_classification_report.txt
Confusion matrix saved to results/sentiment_analysis_experiment/metrics\february_confusion_matrix.png
Classification report saved to results/sentiment_analysis_experiment/metrics\march_classification_report.txt
Confusion matrix saved to results/sentiment_analysis_experiment/metrics\march_confusion_matrix.png


In [31]:
def filter_positive_negative(df):
    # Exclude neutral instances
    return df[df['label'].isin([1, -1])]

def save_classification_report_excluding_neutral(y_true, y_pred, filepath):
    report = classification_report(
        y_true, 
        y_pred, 
        labels=[1, -1], 
        target_names=["Positive (+1)", "Negative (-1)"]
    )
    with open(filepath, "w") as f:
        f.write(report)
    print(f"Classification report (excluding neutral) saved to {filepath}")

def save_confusion_matrix_excluding_neutral(y_true, y_pred, filepath):
    cm = confusion_matrix(y_true, y_pred, labels=[1, -1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Positive (+1)", "Negative (-1)"])
    disp.plot(cmap="Blues", values_format="d")
    plt.title("Confusion Matrix (Excluding Neutral)")
    plt.savefig(filepath)
    plt.close()
    print(f"Confusion matrix (excluding neutral) saved to {filepath}")

df_jan_filtered = filter_positive_negative(df_jan)
df_feb_filtered = filter_positive_negative(df_feb)
df_march_filtered = filter_positive_negative(df_march)

In [32]:
# January (Excluding Neutral)
save_classification_report_excluding_neutral(
    df_jan_filtered['label'], 
    df_jan_filtered['classification'], 
    os.path.join(output_dir, "january_classification_report_excluding_neutral.txt")
)
save_confusion_matrix_excluding_neutral(
    df_jan_filtered['label'], 
    df_jan_filtered['classification'], 
    os.path.join(output_dir, "january_confusion_matrix_excluding_neutral.png")
)

# February (Excluding Neutral)
save_classification_report_excluding_neutral(
    df_feb_filtered['label'], 
    df_feb_filtered['classification'], 
    os.path.join(output_dir, "february_classification_report_excluding_neutral.txt")
)
save_confusion_matrix_excluding_neutral(
    df_feb_filtered['label'], 
    df_feb_filtered['classification'], 
    os.path.join(output_dir, "february_confusion_matrix_excluding_neutral.png")
)

# March (Excluding Neutral)
save_classification_report_excluding_neutral(
    df_march_filtered['label'], 
    df_march_filtered['classification'], 
    os.path.join(output_dir, "march_classification_report_excluding_neutral.txt")
)
save_confusion_matrix_excluding_neutral(
    df_march_filtered['label'], 
    df_march_filtered['classification'], 
    os.path.join(output_dir, "march_confusion_matrix_excluding_neutral.png")
)

Classification report (excluding neutral) saved to results/sentiment_analysis_experiment/metrics\january_classification_report_excluding_neutral.txt
Confusion matrix (excluding neutral) saved to results/sentiment_analysis_experiment/metrics\january_confusion_matrix_excluding_neutral.png
Classification report (excluding neutral) saved to results/sentiment_analysis_experiment/metrics\february_classification_report_excluding_neutral.txt
Confusion matrix (excluding neutral) saved to results/sentiment_analysis_experiment/metrics\february_confusion_matrix_excluding_neutral.png
Classification report (excluding neutral) saved to results/sentiment_analysis_experiment/metrics\march_classification_report_excluding_neutral.txt
Confusion matrix (excluding neutral) saved to results/sentiment_analysis_experiment/metrics\march_confusion_matrix_excluding_neutral.png


## Embeddings

In [14]:
# Load BERTimbau tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch

def get_bert_embedding(text, tokenizer, model):
    # tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # pass inputs through model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract [CLS] token embedding (shape: [batch_size, hidden_size])
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] token is the first token
    return cls_embedding.squeeze(0).numpy()  # convert to NumPy array

In [16]:
df_jan['embedding'] = df_jan['cleaned_article'].apply(lambda x: get_bert_embedding(x, tokenizer, model))
df_feb['embedding'] = df_feb['cleaned_article'].apply(lambda x: get_bert_embedding(x, tokenizer, model))
df_march['embedding'] = df_march['cleaned_article'].apply(lambda x: get_bert_embedding(x, tokenizer, model))

In [25]:
# Combine January and February data for training
df_train_multi = pd.concat([df_jan, df_feb])
X_train_multi = np.vstack(df_train_multi['embedding'].values)
y_train_multi = df_train_multi['label']

# multi classification and test on march
X_test_multi = np.vstack(df_march['embedding'].values)
y_test_multi = df_march['label']

# binary Classification and test on march
df_train_binary = df_train_multi[df_train_multi['label'] != 0]
df_march_binary = df_march[df_march['label'] != 0]

X_train_binary = np.vstack(df_train_binary['embedding'].values)
y_train_binary = df_train_binary['label']

X_test_binary = np.vstack(df_march_binary['embedding'].values)
y_test_binary = df_march_binary['label']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Train multi-class classifier
multi_clf = LogisticRegression(max_iter=1000, class_weight='balanced')
multi_clf.fit(X_train_multi, y_train_multi)

# Predict and evaluate
y_pred_multi = multi_clf.predict(X_test_multi)
print("Multi-Class Report:\n", classification_report(y_test_multi, y_pred_multi))

In [ ]:
# Train binary classifier
binary_clf = LogisticRegression(max_iter=1000, class_weight='balanced')
binary_clf.fit(X_train_binary, y_train_binary)

# Predict and evaluate
y_pred_binary = binary_clf.predict(X_test_binary)
print("Binary Classification Report:\n", classification_report(y_test_binary, y_pred_binary))

In [ ]:
import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

results_dir = "results/bert_embeddings_experiment_v1/metrics"
os.makedirs(results_dir, exist_ok=True)

cm_multi = confusion_matrix(y_test_multi, y_pred_multi)
class_report = classification_report(y_test_multi, y_pred_multi, target_names=["-1", "0", "1"])

report_path = os.path.join(results_dir, "classification_report_multi.txt")
with open(report_path, "w") as f:
    f.write("Classification Report:\n")
    f.write(class_report)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_multi, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["-1", "0", "1"], yticklabels=["-1", "0", "1"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Multi-Class Confusion Matrix")

conf_matrix_path = os.path.join(results_dir, "confusion_matrix_multi.png")
plt.savefig(conf_matrix_path)
plt.close()

print(f"Classification report saved to: {report_path}")
print(f"Confusion matrix saved to: {conf_matrix_path}")

In [ ]:
cm_binary = confusion_matrix(y_test_binary, y_pred_binary)
class_report_binary = classification_report(y_test_binary, y_pred_binary, target_names=["-1", "1"])

report_path_binary = os.path.join(results_dir, "classification_report_binary.txt")
with open(report_path_binary, "w") as f:
    f.write("Classification Report (Binary):\n")
    f.write(class_report_binary)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_binary, annot=True, fmt="d", cmap="Greens", 
            xticklabels=["-1", "1"], yticklabels=["-1", "1"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Binary Confusion Matrix")

conf_matrix_path_binary = os.path.join(results_dir, "confusion_matrix_binary.png")
plt.savefig(conf_matrix_path_binary)
plt.close()

print(f"Binary classification report saved to: {report_path_binary}")
print(f"Binary confusion matrix saved to: {conf_matrix_path_binary}")